In [2]:
import pandas
import numpy
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from random import shuffle


In [5]:
# table = list([float(_) for _ in row] for row in csv.reader(open("ecg.csv", "r")))
# normal = list(map(lambda row: row[:-1], filter(lambda row: row[-1] == 0, table)))
# abnormal = list(map(lambda row: row[:-1], filter(lambda row: row[-1] == 1, table)))
# dataset = [*[(0, _) for _ in normal], *[(1, _) for _ in abnormal]]
# shuffle(dataset)


In [3]:
df = pandas.read_csv("ecg.csv", header=None)

In [45]:
class Block(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Block, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.c2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden_state):
        y = torch.cat((x, hidden_state), 1).float()
        y = self.i2h(y)
        h = torch.sigmoid(y)
        y = self.c2o(h)
        return y, h

    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))


In [46]:
def ResampleLinear1D(original, targetLen):
    original = numpy.array(original, dtype=numpy.float32)
    index_arr = numpy.linspace(0, len(original) - 1, num=targetLen, dtype=numpy.float32)
    index_floor = numpy.array(index_arr, dtype=numpy.int64)
    index_ceil = index_floor + 1
    index_rem = index_arr - index_floor
    val1 = original[index_floor]
    val2 = original[index_ceil % len(original)]
    interp = val1 * (1.0 - index_rem) + val2 * index_rem
    assert len(interp) == targetLen
    return interp


In [55]:
EPOCHS = 1
RESAMPLING_SIZE = 100

model = Block(RESAMPLING_SIZE, RESAMPLING_SIZE, 1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
float()

for epoch in range(EPOCHS):
    running_loss = 0.0
    hidden_state = model.init_hidden()
    for i, (*inputs, label) in enumerate(df.values):
        label = int(label)
        inputs = ResampleLinear1D(inputs, RESAMPLING_SIZE)
        optimizer.zero_grad()
        outputs, hidden_state = model(torch.tensor([inputs]), hidden_state)
        loss = criterion(outputs, torch.tensor([label]))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
        break


IndexError: Target 1 is out of bounds.